In [1]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

def setup_environment():
    """Set up API KEYs."""
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key = getpass.getpass("Enter your OpenAI API key")
        os.environ["OPENAI_API_KEY"] = openai_key

    if not os.environ.get("LANGSMITH_API_KEY"):
        langsmith_key = getpass.getpass("Enter your LangSmith API key")
        if langsmith_key:
               os.environ["LANGSMITH_API_KEY"] = langsmith_key
               os.environ["LANGCHAIN_TRACING_V2"] = "true"
               os.environ["LANGSMITH_PROJECT"] = "Langgraph tutotrial"
        else:
             print("Skipping LangSmith set up")
    
    print("Environment setup completed")


In [2]:
setup_environment()

Environment setup completed


In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

class State(TypedDict):
    # Messages will store the conversation history
    ### add_messages is a special function that append new message instead of replacing them

    messages: Annotated[List, add_messages]
    

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
)
llm.invoke("Hi").content

'Hello! How can I assist you today?'

In [5]:
print(f"Model: {llm.model_name}")

Model: gpt-4o


In [6]:
def chatbot_node(state: State) -> Dict[str, Any]:
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

In [7]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

simple_chatbot = graph_builder.compile()

In [8]:
initial_state = {
    "messages": [HumanMessage(content="Hello! My name is Niyantarana. What is your name?")]
}
result = simple_chatbot.invoke(initial_state)

In [9]:
result['messages']

[HumanMessage(content='Hello! My name is Niyantarana. What is your name?', additional_kwargs={}, response_metadata={}, id='03e9685a-df96-4bed-a4a3-263bb76c8ebe'),
 AIMessage(content='Hello Niyantarana! I’m an AI language model created by OpenAI, and I don’t have a personal name. You can call me Assistant if you’d like. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 22, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BnieggVGEXk51PctVUK8hTnmD0TlF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--203107e5-20bf-4f28-b284-77d5a27d8d1b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 43, 'tota

In [10]:
for i, message in enumerate(result['messages']):
    if isinstance(message, HumanMessage):
        print(f"User: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(f"Unknown message type at index {i}: {message}")

User: Hello! My name is Niyantarana. What is your name?
AI: Hello Niyantarana! I’m an AI language model created by OpenAI, and I don’t have a personal name. You can call me Assistant if you’d like. How can I assist you today?


In [11]:
def test_simple(user_input):
    initial_state = {
        "messages": [HumanMessage(content=user_input)]
    }
    result = simple_chatbot.invoke(initial_state)
    
    for i, message in enumerate(result['messages']):
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"AI: {message.content}")
        else:
            print(f"Unknown message type at index {i}: {message}")

In [12]:
test_simple("What is the GATE cut off for CSE?")

Human: What is the GATE cut off for CSE?
AI: The GATE (Graduate Aptitude Test in Engineering) cut-off for Computer Science Engineering (CSE) varies each year, depending on various factors such as the difficulty level of the exam, the number of candidates appearing for the exam, and the number of available seats in the institutions. The cut-off marks are different for different categories like General, OBC, SC, ST, and PwD.

For the most accurate and up-to-date information, you should check the official GATE website or the websites of the institutions you are interested in. They typically publish the cut-off scores after the results are announced.


In [13]:
test_simple("What are the Subjects in GATE?")

Human: What are the Subjects in GATE?
AI: The Graduate Aptitude Test in Engineering (GATE) is an examination that primarily tests the comprehensive understanding of various undergraduate subjects in engineering and science for admission into master's programs and for recruitment by some public sector companies. As of the latest updates, here are the subjects (referred to as papers) in which GATE is conducted:

1. Aerospace Engineering (AE)
2. Agricultural Engineering (AG)
3. Architecture and Planning (AR)
4. Biomedical Engineering (BM)
5. Biotechnology (BT)
6. Civil Engineering (CE)
7. Chemical Engineering (CH)
8. Computer Science and Information Technology (CS)
9. Chemistry (CY)
10. Electronics and Communication Engineering (EC)
11. Electrical Engineering (EE)
12. Environmental Science and Engineering (ES)
13. Ecology and Evolution (EY)
14. Geology and Geophysics (GG)
15. Instrumentation Engineering (IN)
16. Mathematics (MA)
17. Mechanical Engineering (ME)
18. Mining Engineering (MN)


In [14]:
test_simple("What is the NET cut off for CSE?")

Human: What is the NET cut off for CSE?
AI: The NET (National Eligibility Test) cut-off for Computer Science and Electronics (CSE) can vary each year based on factors such as the number of candidates, difficulty level of the exam, and overall performance. The cut-off marks are generally released by the conducting body, which is the National Testing Agency (NTA) in India, along with or shortly after the results are announced.

To find the exact cut-off for a specific year, you would need to check the official NTA website or the official notification related to the NET exam results for that year. They usually publish the cut-off marks for each subject and category (General, OBC, SC, ST, etc.) on their official platform.


In [15]:
test_simple("What was my last question?")

Human: What was my last question?
AI: I'm sorry, but I don't have the ability to recall past interactions or previous questions. My design is focused on maintaining privacy and confidentiality, which means I don't store personal data or past conversations. How can I assist you today?


In [16]:
!pip install langgraph-checkpoint-sqlite

In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver.from_conn_string(":memory")

In [18]:
graph_builder_with_memory = StateGraph(State)

graph_builder_with_memory.add_node("chatbot", chatbot_node)
graph_builder_with_memory.add_edge(START, "chatbot")
graph_builder_with_memory.add_edge("chatbot", END)

chatbot_with_memory = graph_builder_with_memory.compile(checkpointer=checkpointer)


In [19]:
import uuid
unique_id = uuid.uuid4()
str(unique_id)
print(f"Unique ID for this conversation: {unique_id}")

Unique ID for this conversation: 7514eeb5-3189-49ab-a754-826423eabd40


In [23]:
unique_id = "7514eeb5-3189-49ab-a754-826423eabd40"

def test_simple(user_input):
    config = {"configurable": {"thread_id": str(unique_id)}}
    result = chatbot_with_memory.invoke(
        {
            "messages": [HumanMessage(content=user_input)],
            "configurable": config
        }
    )   
    return result
    

In [24]:
def test_memory_chatbot():
    print("🧠 Testing Chatbot with Memory")
    print("=" * 50)
    
    # Configuration with thread_id - this is crucial for memory!
    config = {"configurable": {"thread_id": "conversation_1"}}
    
    # First message
    print("📝 First interaction:")
    result1 = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content="Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.")]},
        config  # Pass config as second parameter!
    )
    
    for message in result1["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n📝 Second interaction (same thread_id):")
    result2 = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content="What's my name and what I do?")]},
        config  # Same config = same memory!
    )
    
    # Only show the new messages
    new_messages = result2["messages"][len(result1["messages"]):]
    for message in new_messages:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n✅ Success! The chatbot remembered both your name and Specialization!")
    return result2

# Test the memory
memory_result = test_memory_chatbot()


🧠 Testing Chatbot with Memory
📝 First interaction:


AttributeError: '_GeneratorContextManager' object has no attribute 'get_next_version'